# A/B-тестирование

**Автор** Пинчук Ольга

## 1. Предобработка данных
- изучить общую информацию о данных
- проверить пропуски и обработать
- проверить соответствие типов данных
- скорректировать название столбцов
- проверить на дубликаты и обработать 

In [6]:
# Импорт требуемых библиотек
import pandas as pd
import numpy as np
import datetime as dt
from scipy import stats as st
import plotly.graph_objects as go
import math as mth
import plotly.io as pio

import warnings
warnings.simplefilter('ignore')

In [7]:
# функция вывода результатов
def print_some(name):
    print(name)
    df_dict[name].info()   
    display(df_dict[name].head())
    print()  

In [8]:
# Загружаем данные
df_list = ['ab_project_marketing_events','final_ab_events','final_ab_new_users','final_ab_participants']
df_dict = {}
for d in df_list:
    try:
        df_dict[d] = pd.read_csv(f'datasets/{d}.csv')  # Локальный путь
    except:
        df_dict[d] = pd.read_csv(f'/datasets/{d}.csv')  # Серверный путь
    print_some(d)

ab_project_marketing_events
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11



final_ab_events
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99



final_ab_new_users
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone



final_ab_participants
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


Проверка типов данных:
**ab_project_marketing_events**
- start_dt и finish_dt- нужно приводить к типу дата

**final_ab_events**
- event_dt- нужно приводить к типу датавремя
- в details обнаружены пропуски- нужно проверить их природу

**final_ab_new_users**
- first_date  - нужно приводить к типу дата

В остальном типы данных соответствуют данным
Все значения в строках приводим к нижнему регистру для дальнейшего корректного поиска дубликатов


In [9]:
# Приведение всего к нижнему регистру
for d in df_list:
    for col in df_dict[d]:
        try:
            df_dict[d][col] = df_dict[d][col].str.lower()
    
        except: pass
    print(d)
    display(df_dict[d].head())
        

ab_project_marketing_events


,name,regions,start_dt,finish_dt
0,christmas&new year promo,"eu, n.america",2020-12-25,2021-01-03
1,st. valentine's day giveaway,"eu, cis, apac, n.america",2020-02-14,2020-02-16
2,st. patric's day promo,"eu, n.america",2020-03-17,2020-03-19
3,easter promo,"eu, cis, apac, n.america",2020-04-12,2020-04-19
4,4th of july promo,n.america,2020-07-04,2020-07-11


final_ab_events


,user_id,event_dt,event_name,details
0,e1bddce0dafa2679,2020-12-07 20:22:03,purchase,99.99
1,7b6452f081f49504,2020-12-07 09:22:53,purchase,9.99
2,9cd9f34546df254c,2020-12-07 12:59:29,purchase,4.99
3,96f27a054b191457,2020-12-07 04:02:40,purchase,4.99
4,1fd7660fdf94ca1f,2020-12-07 10:15:09,purchase,4.99


final_ab_new_users


,user_id,first_date,region,device
0,d72a72121175d8be,2020-12-07,eu,pc
1,f1c668619dfe6e65,2020-12-07,n.america,android
2,2e1bf1d4c37ea01f,2020-12-07,eu,pc
3,50734a22c0c63768,2020-12-07,eu,iphone
4,e1bddce0dafa2679,2020-12-07,n.america,iphone


final_ab_participants


,user_id,group,ab_test
0,d1aba3e2887b6a73,a,recommender_system_test
1,a7a3664bd6242119,a,recommender_system_test
2,dabc14fddfadd29e,a,recommender_system_test
3,04988c5df189632e,a,recommender_system_test
4,482f14783456d21b,b,recommender_system_test


In [10]:
# замена типов данных
df_dict['ab_project_marketing_events']['start_dt'] = df_dict['ab_project_marketing_events']['start_dt'].map(
    lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df_dict['ab_project_marketing_events']['finish_dt'] = df_dict['ab_project_marketing_events']['finish_dt'].map(
    lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df_dict['final_ab_events']['event_dt'] = df_dict['final_ab_events']['event_dt'].map(
    lambda x: dt.datetime.strptime(x, '%Y-%m-%d %X'))
df_dict['final_ab_new_users']['first_date'] = df_dict['final_ab_new_users']['first_date'].map(
    lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

for d in df_list:
    print(d)
    display(df_dict[d].info())
    print()

ab_project_marketing_events
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes


None


final_ab_events
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     440317 non-null  object        
 1   event_dt    440317 non-null  datetime64[ns]
 2   event_name  440317 non-null  object        
 3   details     62740 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 13.4+ MB


None


final_ab_new_users
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     61733 non-null  object        
 1   first_date  61733 non-null  datetime64[ns]
 2   region      61733 non-null  object        
 3   device      61733 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.9+ MB


None


final_ab_participants
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


None

In [11]:
# Поиск дубликатов
for d in df_list:
    print(d)
    print(df_dict[d].duplicated().sum())

ab_project_marketing_events
0
final_ab_events
0
final_ab_new_users
0
final_ab_participants
0


In [12]:
df_dict['final_ab_events'][df_dict['final_ab_events']['details'].isna()].groupby('event_name').count()

,user_id,event_dt,details
event_name,,,
login,189552,189552,0
product_cart,62462,62462,0
product_page,125563,125563,0


In [13]:
df_dict['final_ab_events'].groupby('event_name')['details'].count()

event_name
login               0
product_cart        0
product_page        0
purchase        62740
Name: details, dtype: int64

### Результат предобработки
- Типы данных приведены к требуемым
- Дубликаты в данных не обнаружены
- Обнаружены пропуски в датафрейме final_ab_events столбец details. В данном поле есть значения только для покупок и содержат их стоимость

## 2. Исследовательский анализ данных
- Как меняется конверсия в воронке на разных этапах?
- Количество событий на пользователя одинаково распределены в выборках?
- В выборках встречаются одни и те же пользователи?
- Как число событий распределено по дням?
- Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?

In [14]:
df_dict['final_ab_events'][~df_dict['final_ab_events']['user_id'].isin(df_dict['final_ab_new_users']['user_id'].unique())]


,user_id,event_dt,event_name,details


В файле с действиями 'final_ab_events' только данные пользователей зарегистрировавшиеся с 7 по 21 декабря 2020 года

In [15]:
df_dict['final_ab_new_users'][['user_id','first_date']]

,user_id,first_date
0,d72a72121175d8be,2020-12-07
1,f1c668619dfe6e65,2020-12-07
2,2e1bf1d4c37ea01f,2020-12-07
3,50734a22c0c63768,2020-12-07
4,e1bddce0dafa2679,2020-12-07
...,...,...
61728,1db53b933257165d,2020-12-20
61729,538643eb4527ed03,2020-12-20
61730,7adee837d5d8cbbd,2020-12-20
61731,1c7d23927835213f,2020-12-20


In [16]:
# Отбираем данные по интересующему тесту
need_test = df_dict['final_ab_participants'][df_dict['final_ab_participants']['ab_test'] == 'recommender_system_test']
len(need_test)

6701

In [17]:
# Добавляем к действиям дату регистрации и отсеиваем пользователей нужного теста

df_users = df_dict['final_ab_events'].merge(df_dict['final_ab_new_users'][['user_id','first_date','region']], how='left', left_on='user_id', right_on='user_id')
df_users = need_test.merge(df_users, how='left', left_on='user_id', right_on='user_id')
df_users

,user_id,group,ab_test,event_dt,event_name,details,first_date,region
0,d1aba3e2887b6a73,a,recommender_system_test,2020-12-07 14:43:27,purchase,99.99,2020-12-07,eu
1,d1aba3e2887b6a73,a,recommender_system_test,2020-12-25 00:04:56,purchase,4.99,2020-12-07,eu
2,d1aba3e2887b6a73,a,recommender_system_test,2020-12-07 14:43:29,product_cart,NaN,2020-12-07,eu
3,d1aba3e2887b6a73,a,recommender_system_test,2020-12-25 00:04:57,product_cart,NaN,2020-12-07,eu
4,d1aba3e2887b6a73,a,recommender_system_test,2020-12-07 14:43:27,product_page,NaN,2020-12-07,eu
...,...,...,...,...,...,...,...,...
27719,6715343afba285ae,b,recommender_system_test,2020-12-07 10:12:15,login,NaN,2020-12-07,cis
27720,6715343afba285ae,b,recommender_system_test,2020-12-08 22:51:16,login,NaN,2020-12-07,cis
27721,6715343afba285ae,b,recommender_system_test,2020-12-09 02:28:03,login,NaN,2020-12-07,cis
27722,6715343afba285ae,b,recommender_system_test,2020-12-10 22:55:14,login,NaN,2020-12-07,cis


In [18]:
# Воронка
pivot = df_users.pivot_table( values='user_id', index=['event_name'], aggfunc=('count', 'nunique')).sort_values(by='nunique', ascending=False)
pivot =pivot.reindex(['login','product_page','product_cart','purchase']).reset_index()
pivot['part'] = pivot['nunique'] / pivot['count']
pivot

,event_name,count,nunique,part
0,login,11190,3675,0.328418
1,product_page,6930,2303,0.332323
2,product_cart,3247,1079,0.332307
3,purchase,3331,1128,0.338637


In [19]:
# Воронка отрисовка

fig = go.Figure(go.Funnel(
    y = pivot['event_name'],
    x = pivot['nunique'], textinfo = 'value+percent initial'))

fig.update_layout(
                  title="Воронка продаж",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=400,
                  width=900)
fig.show()



In [21]:
#pio.write_html(fig, file='figure1.html', auto_open=True)

[картинка](12_figure1) 

@@include[12_figure1](includes/12_figure1.html) 

Конверсия из авторизации в заход на строницу продукта - 63%,
Со страницы продукта в корзину конверсия 50%,
Из авторизации в покупку- 31%

In [16]:
#Распределение пользователей по группам тестирования
print('Всего пользователей:', df_users['user_id'].nunique())
user_A = df_users.query('group == "a"')['user_id'].unique()
print('Пользователей в обеих группах:', df_users.query('user_id in @user_A and group == "b"')['user_id'].nunique())

Всего пользователей: 6701
Пользователей в обеих группах: 0


Пересечения пользователей нет

In [17]:
#Кол-во событий на пользователя в выборках
events_user = df_users.groupby(['group','event_name'])['user_id'].agg([ 'nunique', 'count'])
events_user['events_per_user'] = events_user['count'] /events_user['nunique']
events_user

nunique  count  events_per_user
group event_name                                   
a     login            2747   8566         3.118311
      product_cart      824   2558         3.104369
      product_page     1780   5525         3.103933
      purchase          872   2655         3.044725
b     login             928   2624         2.827586
      product_cart      255    689         2.701961
      product_page      523   1405         2.686424
      purchase          256    676         2.640625

In [18]:
#Распределение событий по дням
df_users['day'] = df_users['event_dt'].dt.date
per_day = df_users.groupby(['day','group'])['event_name'].count().reset_index()
per_day.columns = ['day','group','cnt_event']


In [19]:
#График распределения событий по дням
a = per_day[per_day['group'] == 'a']
b = per_day[per_day['group'] == 'b']
fig = go.Figure()
fig.add_trace(go.Scatter(x=a['day'], y=a['cnt_event'], name = 'group A'))
fig.add_trace(go.Scatter(x=b['day'], y=b['cnt_event'], name = 'group B'))

fig.update_layout(
                  title="Распределение событий по дням",
                  xaxis_title="Дата",
                  yaxis_title="Кол-во событий",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=320,
                  width=900)
fig.show()

с 13 по 21 декабря наблюдается рост по кол-ву событий и далее спад


In [20]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27724 entries, 0 to 27723
Data columns (total 9 columns):
user_id       27724 non-null object
group         27724 non-null object
ab_test       27724 non-null object
event_dt      24698 non-null datetime64[ns]
event_name    24698 non-null object
details       3331 non-null float64
first_date    24698 non-null datetime64[ns]
region        24698 non-null object
day           24698 non-null object
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 3.4+ MB


In [21]:
#Время теста
print(df_users['event_dt'].min())
print(df_users['event_dt'].max())

2020-12-07 00:05:57
2020-12-30 12:42:57


In [22]:
#Время теста и маркетинговые события
df_dict['ab_project_marketing_events'][
    (df_dict['ab_project_marketing_events']['start_dt'].dt.date >= df_users['event_dt'].min())
     & (df_dict['ab_project_marketing_events']['start_dt'].dt.date <= df_users['event_dt'].max()) 
     |
     (df_dict['ab_project_marketing_events']['finish_dt'].dt.date >= df_users['event_dt'].min()) &
    (df_dict['ab_project_marketing_events']['finish_dt'].dt.date <= df_users['event_dt'].max())
     ]

,name,regions,start_dt,finish_dt
0,christmas&new year promo,"eu, n.america",2020-12-25,2021-01-03
10,cis new year gift lottery,cis,2020-12-30,2021-01-07


В период проведения теста проводилось 2 маркетинговых мероприятия 'christmas&new year promo' и 'cis new year gift lottery'.
'cis new year gift lottery' - проводится в другом регионе, по этому его можно не учитывать

**Не пересекалась ли аудитория теста с конкурирующими тестами**

In [23]:
user_un =df_users['user_id'].unique()
other = df_dict['final_ab_participants'][df_dict['final_ab_participants']['ab_test'] != 'recommender_system_test'].query('user_id in @user_un')
print(len(other))
other['ab_test'].unique()

1602


array(['interface_eu_test'], dtype=object)

**Распределение пользователей интересующего теста по регионам**

In [24]:
#График распределения событий по дням

fig = go.Figure()

fig.add_trace(go.Histogram(x=df_users[df_users['group'] == 'a']['region'], name = 'group A'))
fig.add_trace(go.Histogram(x=df_users[df_users['group'] == 'b']['region'], name = 'group B'))

fig.update_layout(
                  title="Распределение событий по дням",
                  xaxis_title="Дата",
                  yaxis_title="Кол-во событий",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=320,
                  width=900)
fig.show()

In [25]:
df_users['region'] = df_users['region'].fillna('not new user')
regions = df_users.groupby('region')['user_id'].nunique().reset_index()
regions['part'] = regions['user_id'] / df_users['user_id'].nunique()
regions

,region,user_id,part
0,apac,45,0.006715
1,cis,30,0.004477
2,eu,3481,0.519475
3,n.america,119,0.017759
4,not new user,3026,0.451574


In [26]:
regions['part'] = regions['user_id'] / df_users[df_users['region'] != 'not new user']['user_id'].nunique()
regions[regions['region'] != 'not new user']

,region,user_id,part
0,apac,45,0.012245
1,cis,30,0.008163
2,eu,3481,0.947211
3,n.america,119,0.032381


Из всех пользователей попавших в интересующий нас тест 55% из которых 94% из региона EU

### Выводы
1. Конверсия из авторизации в заход на строницу продукта - 66%, cо страницы продукта в покупку конверсия 50%, из авторизации в покупку- 33%.
2. В группе A в среднем 3 событиz на пользователя, а в группе В- 2 события
3. С 13 по 21 декабря наблюдается рост по кол-ву событий и далее спад
4. В период проведения теста проводилось 2 маркетинговых мероприятия 'christmas&new year promo' и 'cis new year gift lottery'
5. 887 пользователей из попавших в наш тест так же приняли участие в конкурируещем тесте 'interface_eu_test'
6. 51% новых пользователей в тесте из региона EU

In [40]:
# Код ревьюера
3481 / df_dict['final_ab_new_users'].query('region == "eu"')['user_id'].nunique()

0.07523233196455587

## 3. Оценка результатов A/B-тестирования
Ожидаемый результат:
- аудитория: 15% новых пользователей из региона EU;
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- ожидаемое количество участников теста: 6000.
- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    - конверсии в просмотр карточек товаров — событие `product_page`,
    - просмотры корзины — `product_cart`,
    - покупки — `purchase`.

Проанализируйте результаты теста.

- Что можно сказать про результаты A/В-тестирования?
- Проверьте статистическую разницу долей z-критерием.


In [27]:
# Разделяем данные по интересующим событиям и группам тестирования
product_pageA = df_users[(df_users['event_name'] == 'product_page') & (df_users['group'] == 'a')]
product_pageB = df_users[(df_users['event_name'] == 'product_page') & (df_users['group'] == 'b')]

product_cartA = df_users[(df_users['event_name'] == 'product_cart') & (df_users['group'] == 'a')]
product_cartB = df_users[(df_users['event_name'] == 'product_cart') & (df_users['group'] == 'b')]

purchaseA = df_users[(df_users['event_name'] == 'purchase') & (df_users['group'] == 'a')]
purchaseB = df_users[(df_users['event_name'] == 'purchase') & (df_users['group'] == 'b')]

events_df =[product_pageA, product_pageB, product_cartA, product_cartB, purchaseA, purchaseB]
for d in events_df:
    print(len(d))

5525
1405
2558
689
2655
676


In [28]:
login_per_day = df_users[df_users['event_name'] == 'login']
login_per_day = login_per_day.groupby(['group','day'])['user_id'].nunique().reset_index()
login_per_day = login_per_day.rename(columns ={'user_id': 'cnt_users_login'})


In [29]:
# группируем по дням событий и считаем ежедневную конверсию

product_pageA_daily = product_pageA.groupby('day')['user_id'].nunique().reset_index()
product_pageA_daily.columns =['day', 'cnt_product_pageA']
product_pageA_daily = product_pageA_daily.merge(login_per_day[login_per_day['group'] =='a'].drop('group', axis=1), how ='left', left_on='day', right_on='day')
product_pageA_daily['convers'] = product_pageA_daily['cnt_product_pageA']/product_pageA_daily['cnt_users_login']
product_pageA_daily

product_pageB_daily = product_pageB.groupby('day')['user_id'].nunique().reset_index()
product_pageB_daily.columns =['day', 'cnt_product_pageB']
product_pageB_daily = product_pageB_daily.merge(login_per_day[login_per_day['group'] =='b'].drop('group', axis=1), left_on='day', right_on='day')
product_pageB_daily['convers'] = product_pageB_daily['cnt_product_pageB']/product_pageB_daily['cnt_users_login'] 

product_cartA_daily = product_cartA.groupby('day')['user_id'].nunique().reset_index()
product_cartA_daily.columns =['day', 'cnt_product_cartA']
product_cartA_daily = product_cartA_daily.merge(login_per_day[login_per_day['group'] =='a'].drop('group', axis=1), left_on='day', right_on='day')
product_cartA_daily['convers'] = product_cartA_daily['cnt_product_cartA']/product_cartA_daily['cnt_users_login'] 

product_cartB_daily = product_cartB.groupby('day')['user_id'].nunique().reset_index()
product_cartB_daily.columns =['day', 'cnt_product_cartB']
product_cartB_daily = product_cartB_daily.merge(login_per_day[login_per_day['group'] =='b'].drop('group', axis=1), left_on='day', right_on='day')
product_cartB_daily['convers'] = product_cartB_daily['cnt_product_cartB']/product_cartB_daily['cnt_users_login'] 


purchaseA_daily = purchaseA.groupby('day')['user_id'].nunique().reset_index()
purchaseA_daily.columns =['day', 'cnt_purchaseA']
purchaseA_daily = purchaseA_daily.merge(login_per_day[login_per_day['group'] =='a'].drop('group', axis=1), left_on='day', right_on='day')
purchaseA_daily['convers'] = purchaseA_daily['cnt_purchaseA']/purchaseA_daily['cnt_users_login']


purchaseB_daily = purchaseB.groupby('day')['user_id'].nunique().reset_index()
purchaseB_daily.columns =['day', 'cnt_purchaseB']
purchaseB_daily = purchaseB_daily.merge(login_per_day[login_per_day['group'] =='b'].drop('group', axis=1), left_on='day', right_on='day')
purchaseB_daily['convers'] = purchaseB_daily['cnt_purchaseB']/purchaseB_daily['cnt_users_login']


In [30]:
# Графики конверсии по дням
def my_plot(A, B, name):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=A['day'], y=A['convers'], name= 'A'))
    fig.add_trace(go.Scatter(x=B['day'], y=B['convers'], name= 'B'))

    fig.update_layout(
                    title=name,
                    xaxis_title="Дата",
                    yaxis_title="Кол-во событий",
                    margin=dict(l=0, r=0, t=30, b=0),
                    height=320,
                  width=900)
    fig.show()

In [31]:
my_plot(product_pageA_daily, product_pageB_daily,'Конверсия в просмотр карточек товаров' )

In [32]:
my_plot(product_cartA_daily, product_cartB_daily,'Конверсия в просмотр корзины' )

In [33]:
my_plot(purchaseA_daily, purchaseB_daily,'Конверсия покупок' )



Изменения в конверсии:
- Конверсия в просмотр карточек товаров: Группа В показывает стабильно более низкие показатели, чем группа А
- Конверсия в просмотр корзины: 30 декабря группа В показывает значительный скачек в просмотре корзины, но так как нет данных по этому дню в группе А, сказать что-то определенное нельзя, возможно это нормальное поведение перед праздником. В первые 9 дней теста группа В соревновалась с группой А, но далее показывает более низкие значения конверсии
- Конверсии покупок: с 14 до 29 декабря группа В показывает стабильно более низкие значения в кол-ве покупок

In [34]:

def test_z(alpha,successesA,successesB,trialsA,trialsB):
    
    # пропорция успехов в первой группе:
    p1 = successesA/trialsA

    # пропорция успехов во второй группе:
    p2 = successesB/trialsB

    # пропорция успехов в комбинированном датасете:
    p_combined = (successesA + successesB) / (trialsA + trialsB)

    # разница пропорций в датасетах
    difference = p1 - p2 

    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / trialsA + 1 / trialsB))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)

    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-значение: ', p_value)

    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли в разными') 

In [35]:
print('Проверка статистической разницы долей z-критерием:')
print('   Просмотр карточек товаров:')
test_z(0.05,product_pageA_daily['cnt_product_pageA'].sum(),product_pageB_daily['cnt_product_pageB'].sum(),product_pageA_daily['cnt_users_login'].sum(),product_pageB_daily['cnt_users_login'].sum())
print('   Просмотр корзины:')
test_z(0.05,product_cartA_daily['cnt_product_cartA'].sum(),product_cartB_daily['cnt_product_cartB'].sum(),product_cartA_daily['cnt_users_login'].sum(),product_cartB_daily['cnt_users_login'].sum())

print('   Покупки:')
test_z(0.05,purchaseA_daily['cnt_purchaseA'].sum(),purchaseB_daily['cnt_purchaseB'].sum(),purchaseA_daily['cnt_users_login'].sum(),purchaseB_daily['cnt_users_login'].sum())

Проверка статистической разницы долей z-критерием:
   Просмотр карточек товаров:
p-значение:  0.0
Отвергаем нулевую гипотезу: между долями есть значимая разница
   Просмотр корзины:
p-значение:  0.00037128846600120724
Отвергаем нулевую гипотезу: между долями есть значимая разница
   Покупки:
p-значение:  3.258956398077828e-07
Отвергаем нулевую гипотезу: между долями есть значимая разница


## Общий вывод
Опишите выводы по этапу исследовательского анализа данных и по проведённой оценке результатов A/B-тестирования.

1. Конверсия из авторизации в заход на строницу продукта - 66%, cо страницы продукта в покупку конверсия 50%, из авторизации в покупку- 33%.
2. В группе A в среднем 6 событий на пользователя, а в группе В- 5 событий
3. С 13 по 21 декабря наблюдается рост по кол-ву событий и далее спад
4. В период проведения теста проводилось маркетинговое мероприятие 'christmas&new year promo' 
5. 887 пользователей из попавших в наш тест так же приняли участие в конкурируещем тесте 'interface_eu_test'
6. 52% новых пользователей в тесте из региона EU

**Различие конверсии по группам теста:**
- Конверсия в просмотр карточек товаров: Группа В показывает стабильно более низкие показатели, чем группа А
- Конверсия в просмотр корзины: 30 декабря группа В показывает значительный скачек в просмотре корзины, но так как нет данных по этому дню в группе А, сказать что-то определенное нельзя, возможно это нормальное поведение перед праздником. В первые 9 дней теста группа В соревновалась с группой А, но далее показывает более низкие значения конверсии
- Конверсии покупок: с 14 до 29 декабря группа В показывает стабильно более низкие значения в кол-ве покупок



Проверка статистической разницы долей z-критерием показала наличие разницы между долями

По данным конверсии группа В показывает более низкие значения, что говорит о том что тест прошел не успешно, но т.к. в период проведения теста проводилось 2 маркетинговых мероприятия и конкурирующий тест - нельзя определенно сказать, что полученные результаты являются следствием только внедрениея улучшенной рекомендательной системы.

Рекомендую в период проведения маркетинговых мероприятий не проводить тестирование нового функционала, для чистоты данных. Кроме того проведение теста в период маркетинговых мероприятий может негаливно сказаться на результаты маркетингового мероприятия, например сократить приток клиентов и снизить выручку.